# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np
from abstract_matrix import AbstractMatrix, array_determinant, array_cofactor

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

# ex = alg.Examples(alg_dir)

In [2]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4578851920
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

In [3]:
mat1 = AbstractMatrix.zeros((2, 3), ps3)

mat1

PSRing3 Matrix:
[['{}' '{}' '{}']
 ['{}' '{}' '{}']]

In [4]:
fubar = [['{0}', '{1}', '{2}'],
         ['{2}', '{1}', '{0}']]

mat2 = AbstractMatrix(fubar, ps3)

mat2

PSRing3 Matrix:
[['{0}' '{1}' '{2}']
 ['{2}' '{1}' '{0}']]

In [5]:
mat3 = mat2.transpose()
mat3

PSRing3 Matrix:
[['{0}' '{2}']
 ['{1}' '{1}']
 ['{2}' '{0}']]

In [6]:
mat2x3 = mat2 * mat3
mat2x3

PSRing3 Matrix:
[['{0, 1, 2}' '{1}']
 ['{1}' '{0, 1, 2}']]

In [7]:
mat3x2 = mat3 * mat2
mat3x2

PSRing3 Matrix:
[['{0, 2}' '{}' '{}']
 ['{}' '{}' '{}']
 ['{}' '{}' '{0, 2}']]

In [8]:
try:
    mat2x2 = mat2 * mat2
    print(mat2x2)
except Exception as exc:
    print(exc)

The array shapes are incompatible: 3 columns vs 2 rows


In [9]:
mat2p2 = mat2 + mat2

mat2p2

PSRing3 Matrix:
[['{}' '{}' '{}']
 ['{}' '{}' '{}']]

In [10]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3)
except Exception as exc:
    print(exc)

The array shapes are not equal: (2, 3) != (3, 2)


In [11]:
mat2m2 = mat2 - mat2
mat2

PSRing3 Matrix:
[['{0}' '{1}' '{2}']
 ['{2}' '{1}' '{0}']]

### 2x2 Test

In [12]:
mat2x3

PSRing3 Matrix:
[['{0, 1, 2}' '{1}']
 ['{1}' '{0, 1, 2}']]

In [13]:
array_determinant(mat2x3.array(), mat2x3.ring())

'{0, 2}'

In [14]:
mat2x3.determinant()

'{0, 2}'

In [15]:
ps3.mult('{0, 1, 2}', '{0, 1, 2}')

'{0, 1, 2}'

In [16]:
ps3.mult('{1}', '{1}')

'{1}'

In [17]:
ps3.sub('{0, 1, 2}', '{1}')

'{0, 2}'

In [18]:
array_cofactor(mat2x3.array(), mat2x3.ring())

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

### 3x3 Test

In [19]:
# rnd1 = AbstractMatrix.random((3,3),ps3)
# rnd1.get_array()

In [20]:
rnd1 = np.array([['{1, 2}', '{0, 1, 2}', '{0, 2}'],
                 ['{0, 2}', '{}', '{1}'],
                 ['{0}', '{1}', '{0, 1}']], dtype='<U32')

In [21]:
array_determinant(rnd1, ps3)

'{0, 1}'

In [22]:
rnd1_mat = AbstractMatrix(rnd1, ps3)
rnd1_mat

PSRing3 Matrix:
[['{1, 2}' '{0, 1, 2}' '{0, 2}']
 ['{0, 2}' '{}' '{1}']
 ['{0}' '{1}' '{0, 1}']]

In [23]:
rnd1_mat.determinant()

'{0, 1}'

In [24]:
array_cofactor(rnd1, ps3)

array([['{1}', '{0}', '{}'],
       ['{0, 1}', '{0, 1}', '{0, 1}'],
       ['{1}', '{0, 1, 2}', '{0, 2}']], dtype='<U32')

In [25]:
rnd1_mat_cof = rnd1_mat.cofactor_matrix()
rnd1_mat_cof

PSRing3 Matrix:
[['{1}' '{0}' '{}']
 ['{0, 1}' '{0, 1}' '{0, 1}']
 ['{1}' '{0, 1, 2}' '{0, 2}']]

### 4x4 Test

In [26]:
# rnd2 = AbstractMatrix.random((4,4),ps3)
# rnd2.get_array()

In [27]:
rnd2 = np.array([['{}', '{0, 1, 2}', '{0, 1, 2}', '{}'],
                 ['{0, 1}', '{0, 2}', '{1, 2}', '{2}'],
                 ['{0, 2}', '{}', '{}', '{0, 1}'],
                 ['{1}', '{0}', '{0, 2}', '{}']], dtype='<U32')

In [28]:
array_determinant(rnd2, ps3)

'{1, 2}'

In [29]:
array_cofactor(rnd2, ps3)

array([['{0}', '{0, 1, 2}', '{0}', '{0, 2}'],
       ['{}', '{1}', '{1}', '{2}'],
       ['{2}', '{}', '{}', '{1}'],
       ['{0, 1}', '{0, 1, 2}', '{0, 1, 2}', '{0}']], dtype='<U32')

In [30]:
rnd2_mat = AbstractMatrix(rnd2, ps3)

In [31]:
rnd2_mat.determinant()

'{1, 2}'

In [32]:
rnd2_mat.cofactor_matrix()

PSRing3 Matrix:
[['{0}' '{0, 1, 2}' '{0}' '{0, 2}']
 ['{}' '{1}' '{1}' '{2}']
 ['{2}' '{}' '{}' '{1}']
 ['{0, 1}' '{0, 1, 2}' '{0, 1, 2}' '{0}']]